In [1]:
import torch
from torch_geometric.data import Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import math
import random

In [2]:
df = pd.read_csv("/home/shreyasmn/Downloads/pilDataSets/RAMON_DATASET.csv")
df.iloc[75310,1:13].astype(bool).sum(axis=0)

12

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,rssi_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,...,64.328276,10.0,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0


Adding a deployment column for the dataset

In [8]:
def add_deployment_column(df):
    path = "/home/shreyasmn/Downloads/pilDataSets/RAMON_DATASET.csv"
    count = -1
    
    k=12
    i=0
    bss_num=0
    
    for idx, val in df.iterrows():
        if df.at[idx,"node_type"]==0:
            bss_num+=1
        if (bss_num-1)%k==0 and df.at[idx, "node_type"] == 0:
            count += 1
        df.at[idx, "deployment"] = count
        if bss_num==7680:
            k=10
        if bss_num==14080:
            k=8
    return df


In [9]:
df = add_deployment_column(df)

adding interference map for a given deployment